In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
import os
from utils import BIAS_TYPE,INPUT_TYPE,OUTPUT_TYPE,HIDDEN_TYPE
import time
import flappy_bird_gymnasium
import gymnasium
from config import Config
from observer import Observer
from population import Population


In [ ]:

env = gymnasium.make("FlappyBird-v0")

print(env.action_space.n)
print(env.observation_space.shape[0])


def create_fitness_function(reps = 5):
    def fitness_function(network):
        final = 0
        for _ in range(reps):
            obs, _ = env.reset() 
            cummulative_return = 0
            while True:
                action_values = np.exp(np.array(network.predict(obs)))
                action_values = action_values/np.sum(action_values)
                if(action_values[1] > 0.8):
                    action = 1
                else:
                    action = 0
                
                obs, reward, terminated, _, info = env.step(action)
                
                cummulative_return += np.int32(reward)
                if terminated:
                    break
            final += cummulative_return            
        return final/reps + 1
    return fitness_function

In [ ]:

from utils import COLORS

def plot_best_network(best,rep):
    plt.clf()
    G = nx.MultiDiGraph()
    nn = best.create_network()


    inputs = []
    outputs = []
    forward_pass = []
    backward_pass = []
    for el in nn.inputs:
        G.add_node(el.id,subset=0, color="blue")
        inputs.append(el.id)
        forward_pass.append(el.id)

    aux = []
    for el in nn.outputs:
        G.add_node(el.id,subset=len(nn.layers)+1,color=COLORS[el.activation_function])
        outputs.append(el.id)
        backward_pass.append(el.id)
        aux.append(el)
    
    pos = 0

    while(pos < len(aux)):
        for el in aux[pos].previous:
            aux.append(el)
            if(el.id not in backward_pass):
                backward_pass.append(el.id)
        pos+=1
    
    for id in inputs:
        if(id not in backward_pass):
            backward_pass.append(id)
    
    print(backward_pass)

    for pos in range(len(nn.layers)):
        for el in nn.layers[pos]:
            for node in el.previous:
                if(node.id in forward_pass):
                    forward_pass.append(el.id)
                    break
    for id in outputs:
        if(id not in forward_pass):
            forward_pass.append(id)
    print(forward_pass)



    final = []
    for id in backward_pass:
        if(id in forward_pass):
            final.append(id)
    
    print(final)

    for pos in range(len(nn.layers)):
        for node in nn.layers[pos]:
            print(node)
            if((node.id not in inputs) and (node.id not in outputs)  and (node.id  in final)):
                
                G.add_node(node.id,subset=pos+1,color=COLORS[el.activation_function])

    # for node in best.nodes_genes:
        
    print(G.nodes)
    # for el in c:
    #     nodes.append(el)

    for el in best.connection_genes:
        if(el.is_enabled and el.in_node in final and el.out_node in final):
            G.add_edge(el.in_node,el.out_node)

    

    colors = nx.get_node_attributes(G,'color').values()
    
    pos = nx.multipartite_layout(G)
    # nx.draw_networkx_nodes(G, pos)
    nx.draw(G,pos,arrows = True, with_labels=True, font_weight='bold', node_color=colors)
    plt.savefig(f"Repetition {rep} best")
    plt.clf()

In [ ]:

path = "fitnesses/"
repetitions = 3
bests = []
for rep in range(1,3):
    print(f"Repetition {rep}:")
    config = Config(env.observation_space.shape[0],env.action_space.n)
    pop=Population(config,fitness_function=create_fitness_function(5))

    observer = Observer()

    pop.train(1000,observer)

    # observers.append(observer)
   
    bests.append(pop.best)
    np.save(f"{path}fit_rep{rep}", np.array(observer.fitnesses))

    plot_best_network(pop.best, rep)
    figure = observer.create_species_plot(pop.species_count)
    figure.savefig(f"Repetition {rep} species")

In [ ]:

def create_fitness_plot(fitnesses, smoothing_window, text =""):
    final = np.array(fitnesses)
    mean = np.mean(final, axis = 0)
    std = np.std(final, axis = 0)
    max_experiment = smooth(mean + std, smoothing_window)
    min_experiment = smooth(mean - std, smoothing_window)
    mean_experiment = smooth(mean, smoothing_window)
    
    

    iterations = [i+1 for i in range(len(mean))]
    plt.xlabel("Epoch")
    plt.ylabel("Fitness")

    plt.ylim([0,50])
    # plt.xticks(iterations)

    plt.plot(iterations, mean_experiment, label = text)
    plt.fill_between(iterations, min_experiment, max_experiment, alpha=.1)
    

def smooth(y, window, poly=1):
    '''
    y: vector to be smoothed 
    window: size of the smoothing window '''
    return savgol_filter(y,window,poly)
            

In [ ]:
figure = plt.figure()
path = "fitnesses_1"
fits = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    # checking if it is a file
    fit = np.load(f)
    for pos in range(1,len(fit)):
        if(fit[pos]< fit[pos-1]):
            fit[pos] = fit[pos-1]
    fits.append(fit)
    print(fit[-1])

create_fitness_plot(fits,20,"hyperparams - 1")

path = "fitnesses_2"
fits = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    # checking if it is a file
    fit = np.load(f)
    for pos in range(1,len(fit)):
        if(fit[pos]< fit[pos-1]):
            fit[pos] = fit[pos-1]
    fits.append(fit)
    print(fit[-1])

create_fitness_plot(fits,20,"hyperparams - 2")
plt.legend()
figure.savefig("results")   